In [137]:
!pip3 install catboost

     |████████████████████████████████| 76.6 MB 2.5 MB/s eta 0:00:01    |██▋                             | 6.3 MB 830 kB/s eta 0:01:25     |███████████████▏                | 36.2 MB 887 kB/s eta 0:00:46     |███████████████████             | 45.7 MB 2.0 MB/s eta 0:00:16
     |████████████████████████████████| 47 kB 550 kB/s eta 0:00:01


In [142]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import ensemble
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import fetch_covtype
from sklearn.ensemble import RandomForestClassifier, RandomForestClassifier
import catboost as cb


warnings.filterwarnings('ignore')

In [120]:
df = pd.read_csv('CrabAgePrediction.csv')

In [121]:
df.isna().sum()

Sex               0
Length            0
Diameter          0
Height            0
Weight            0
Shucked Weight    0
Viscera Weight    0
Shell Weight      0
Age               0
dtype: int64

In [122]:
df.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,F,1.4375,1.1750,0.4125,24.635715,12.332033,5.584852,6.747181,9
1,M,0.8875,0.6500,0.2125,5.400580,2.296310,1.374951,1.559222,6
2,I,1.0375,0.7750,0.2500,7.952035,3.231843,1.601747,2.764076,6
3,F,1.1750,0.8875,0.2500,13.480187,4.748541,2.282135,5.244657,10
4,I,0.8875,0.6625,0.2125,6.903103,3.458639,1.488349,1.700970,6


In [123]:
encoder = preprocessing.LabelEncoder()
df['Sex'] = encoder.fit_transform(df['Sex'])

In [124]:
df.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,1.4375,1.1750,0.4125,24.635715,12.332033,5.584852,6.747181,9
1,2,0.8875,0.6500,0.2125,5.400580,2.296310,1.374951,1.559222,6
2,1,1.0375,0.7750,0.2500,7.952035,3.231843,1.601747,2.764076,6
3,0,1.1750,0.8875,0.2500,13.480187,4.748541,2.282135,5.244657,10
4,1,0.8875,0.6625,0.2125,6.903103,3.458639,1.488349,1.700970,6


In [125]:
X = df.drop(columns=['Age'])
y = df['Age']

In [126]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=271)

In [127]:
x_datasets = np.array_split(x_train, 10)

In [128]:
y_datasets = np.array_split(y_train, 10)

In [129]:
#обучим на каждой подвыборке дерево решений
models = []
for i in range(10):
    model_tree = tree.DecisionTreeRegressor(random_state=0)
    model_tree.fit(x_datasets[i], y_datasets[i])
    models.append(model_tree)

In [130]:
#находим прогноз каждого дерева
y_pred = []
for i in range(len(models)):
    y_pred.append(models[i].predict(x_test))

In [131]:
mean_pred = np.array(y_pred).mean(axis=0)

In [132]:
print(*mean_pred[:10], sep='\t')
print(*y_test[:10], sep='\t')

10.177142857142858	9.289109199978764	13.366666666666665	9.519129179998746	12.284482758620689	17.4	10.457692307692309	10.908125	10.379999999999999	9.997222222222224
10	7	13	10	9	17	10	13	10	10


In [133]:
model_tree = tree.DecisionTreeRegressor(random_state=0)
one_model = model_tree.fit(x_train, y_train)

In [134]:
one_pred = one_model.predict(x_test)

In [135]:
r2_score(mean_pred, y_test)

0.0778722225785321

In [136]:
r2_score(one_pred, y_test)

0.1675078216111735

Boosting

In [139]:
A_train, A_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [140]:
print(A_train.shape)
print(A_test.shape)

(3114, 8)
(779, 8)


In [163]:
random_forest = RandomForestClassifier(max_depth=15,min_samples_split=10).fit(A_train, y_train)
y_preds_d = random_forest.predict(A_train)
print('F1 мера для тренировочных данных',f1_score(y_preds_d,y_train,average='macro'))
y_pred = random_forest.predict(A_test)
print('F1 мера для тестовых данных',f1_score(y_pred,y_test,average = "macro"))

F1 мера для тренировочных данных 0.7572206229645749
F1 мера для тестовых данных 0.12730479892714527


In [165]:
random_forest = RandomForestClassifier()
params_grid = {
    'max_depth': [3, 20],
    'min_samples_leaf': [3, 10],
    'min_samples_split': [6, 12]
}

gcv = GridSearchCV(estimator=random_forest, param_grid=params_grid, scoring='f1_macro', cv = 4)

In [166]:
gcv.fit(A_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 20], 'min_samples_leaf': [3, 10],
                         'min_samples_split': [6, 12]},
             scoring='f1_macro')

In [167]:
best_model = gcv.best_estimator_

In [168]:
y_preds_d = best_model.predict(A_train)
print('F1 мера для тренировочных данных', f1_score(y_preds_d, y_train, average='macro'))

F1 мера для тренировочных данных 0.44432004927400376


In [169]:
y_pred = best_model.predict(A_test)

In [170]:
print('F1 мера для тестовых данных', f1_score(y_pred, y_test, average='macro'))

F1 мера для тестовых данных 0.13125424888664616


In [ ]:
model_catboost_clf = cb.CatBoostClassifier(iterations=3000, task_type='GPU', devices='0')
model_catboost_clf.fit(A_train, y_train)

In [ ]:
y_preds_t = model_catboost_clf.predict(A_train, task_type='GPU')
print('F1 мера для тренировочных данных', f1_score(y_preds_t, y_train, average='macro'))

In [ ]:
y_preds_t = model_catboost_clf.predict(A_test, task_type='GPU')
print('F1 мера для тестовых данных', f1_score(y_preds_t, y_test, average='macro'))